In [39]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
import random
import numpy as np
import statistics

In [40]:
data = pd.read_csv("../Data/Cleaned/from_kickstarter_reduced.csv")

In [41]:
data.describe()

,Unnamed: 0,id,backers_count,goal,usd_pledged,static_usd_rate,year,month,day,hour,days_to_deadline,goal_USD,blurb_length,binary_state
count,20000.000000,2.000000e+04,20000.000000,2.000000e+04,2.000000e+04,20000.000000,20000.00000,20000.000000,20000.000000,20000.0,20000.000000,2.000000e+04,20000.000000,20000.000000
mean,2041.475750,1.075254e+09,152.232300,3.516041e+04,1.267863e+04,1.007821,2015.56540,6.214750,15.108300,0.0,34.265650,2.991567e+04,18.924250,0.404950
std,1179.364453,6.207460e+08,1291.975501,4.486028e+05,8.967284e+04,0.232205,2.16923,3.383387,8.876334,0.0,12.348583,4.393059e+05,5.040167,0.490895
min,0.000000,3.923500e+04,0.000000,1.000000e+00,0.000000e+00,0.008786,2009.00000,1.000000,1.000000,0.0,1.000000,7.356343e-01,1.000000,0.000000
25%,1014.000000,5.357191e+08,2.000000,2.000000e+03,5.327022e+01,1.000000,2014.00000,3.000000,7.000000,0.0,30.000000,2.000000e+03,16.000000,0.000000
50%,2032.000000,1.076134e+09,17.000000,5.000000e+03,9.239855e+02,1.000000,2016.00000,6.000000,15.000000,0.0,30.000000,5.000000e+03,20.000000,0.000000
75%,3055.250000,1.613053e+09,70.000000,1.660775e+04,5.125750e+03,1.000000,2017.00000,9.000000,23.000000,0.0,38.000000,1.500000e+04,22.000000,1.000000
max,4095.000000,2.147431e+09,102203.000000,5.000000e+07,4.148577e+06,1.716408,2019.00000,12.000000,31.000000,0.0,92.000000,5.000000e+07,36.000000,1.000000


In [42]:
num_to_delete = data["binary_state"].value_counts()[0] - data["binary_state"].value_counts()[1]
failed_indices = data[data["binary_state"] == 0].index
i_failed_indices = random.sample(range(0, failed_indices.shape[0]), num_to_delete)
indices_to_drop = []
for index in i_failed_indices:
    indices_to_drop.append(failed_indices[index])

In [43]:
data = data.drop(indices_to_drop)
data.shape

(16198, 30)

In [44]:
data["binary_state"].describe()

count    16198.000000
mean         0.500000
std          0.500015
min          0.000000
25%          0.000000
50%          0.500000
75%          1.000000
max          1.000000
Name: binary_state, dtype: float64

In [45]:
data.columns

Index(['Unnamed: 0', 'id', 'backers_count', 'blurb', 'currency', 'goal',
       'launched_at', 'deadline', 'location.country', 'name', 'usd_pledged',
       'slug', 'spotlight', 'staff_pick', 'static_usd_rate', 'state', 'year',
       'month', 'day', 'hour', 'days_to_deadline', 'goal_USD', 'category_name',
       'category_slug', 'blurb_length', 'location_type', 'location_country',
       'location_state', 'location_displayable_name', 'binary_state'],
      dtype='object')

In [46]:
data.drop(['Unnamed: 0', 'id', 'day', 'hour', 'blurb', 'currency', 'launched_at', 'deadline', 'name', 'slug', 'state', 'location_displayable_name', 'location.country', 'location_state', 'static_usd_rate', 'usd_pledged', 'days_to_deadline'], axis = 1, inplace = True)

In [47]:
data.columns

Index(['backers_count', 'goal', 'spotlight', 'staff_pick', 'year', 'month',
       'goal_USD', 'category_name', 'category_slug', 'blurb_length',
       'location_type', 'location_country', 'binary_state'],
      dtype='object')

In [48]:
categorical_cols = ['location_country', 'category_name', 'category_slug', 'location_type']
binary_categorical = ['spotlish', 'staff_pick']

In [49]:
for cat in categorical_cols:
    print(cat, len(data[cat].unique()))

location_country 115
category_name 159
category_slug 15
location_type 8


In [50]:
data.drop(['location_country', 'category_name'], axis = 1, inplace = True)

In [51]:
data['spotlight'] = data['spotlight'].astype(int)
data['staff_pick'] = data['staff_pick'].astype(int)
data.dtypes

backers_count      int64
goal             float64
spotlight          int32
staff_pick         int32
year               int64
month              int64
goal_USD         float64
category_slug     object
blurb_length       int64
location_type     object
binary_state       int64
dtype: object

In [52]:
data = pd.get_dummies(data)
data.columns

Index(['backers_count', 'goal', 'spotlight', 'staff_pick', 'year', 'month',
       'goal_USD', 'blurb_length', 'binary_state', 'category_slug_art',
       'category_slug_comics', 'category_slug_crafts', 'category_slug_dance',
       'category_slug_design', 'category_slug_fashion',
       'category_slug_film & video', 'category_slug_food',
       'category_slug_games', 'category_slug_journalism',
       'category_slug_music', 'category_slug_photography',
       'category_slug_publishing', 'category_slug_technology',
       'category_slug_theater', 'location_type_Country',
       'location_type_County', 'location_type_Island',
       'location_type_LocalAdmin', 'location_type_Miscellaneous',
       'location_type_Suburb', 'location_type_Town', 'location_type_Zip'],
      dtype='object')

In [53]:
data.shape

(16198, 32)

In [54]:
df_y = data['binary_state']
df_y

2        0
3        0
4        0
5        1
6        1
        ..
19994    1
19995    1
19997    0
19998    0
19999    1
Name: binary_state, Length: 16198, dtype: int64

In [55]:
data.drop(['binary_state'], axis = 1, inplace = True)
data.shape

(16198, 31)

In [56]:
X = data.to_numpy()
y = df_y.to_numpy()

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [58]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(31, input_dim = 31, activation = 'relu'),
  tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [59]:
hist = model.fit(X_train, y_train, epochs = 10, verbose = 1)

Epoch 1/10
12958/12958 [==============================] - 1s 39us/sample - loss: 8.0354 - acc: 0.5015
Epoch 2/10
12958/12958 [==============================] - 0s 31us/sample - loss: 8.0354 - acc: 0.5015
Epoch 3/10
12958/12958 [==============================] - 0s 29us/sample - loss: 8.0354 - acc: 0.5015
Epoch 4/10
12958/12958 [==============================] - 0s 28us/sample - loss: 8.0354 - acc: 0.5015
Epoch 5/10
12958/12958 [==============================] - 0s 31us/sample - loss: 8.0354 - acc: 0.5015
Epoch 6/10
12958/12958 [==============================] - 0s 35us/sample - loss: 8.0354 - acc: 0.5015
Epoch 7/10
12958/12958 [==============================] - 0s 29us/sample - loss: 8.0354 - acc: 0.5015
Epoch 8/10
12958/12958 [==============================] - 0s 30us/sample - loss: 8.0354 - acc: 0.5015
Epoch 9/10
12958/12958 [==============================] - 0s 29us/sample - loss: 8.0354 - acc: 0.5015
Epoch 10/10
12958/12958 [==============================] - 0s 33us/sample - loss: 

In [60]:
model.evaluate(X_test, y_test)

3240/3240 [==============================] - 0s 30us/sample - loss: 8.1436 - acc: 0.4948


[8.143618077407648, 0.4947531]

In [61]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(31, input_dim = 31, activation = 'relu'),
  tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [62]:
def my_cross_val(model, X, y, k):
    foldsX = np.array_split(X, k)
    foldsY = np.array_split(y, k)

    acc = []

    for i in range(0, k):
        testX = foldsX[i]
        testY = foldsY[i]
        train_arrays_X = np.delete(foldsX, i)
        train_arrays_Y = np.delete(foldsY, i)

        trainX = train_arrays_X[0]
        trainY = train_arrays_Y[0]
        for i in range (1, k-1):
            trainX = np.concatenate((trainX, train_arrays_X[i]))
            trainY = np.concatenate((trainY, train_arrays_Y[i]))

        model.fit(trainX, trainY, epochs = 2, verbose = 1)
        ret = model.evaluate(testX, testY)
        acc.append(ret[1])
    mean = 0
    for a in acc:
        mean += a
    mean /= k
    return {"acc": acc, "mean": mean}

In [63]:
ret = my_cross_val(model, X, y, 5)

Epoch 1/2
12958/12958 [==============================] - 1s 46us/sample - loss: 9.0744 - acc: 0.4309
Epoch 2/2
3240/3240 [==============================] - 0s 28us/sample - loss: 7.9318 - acc: 0.5025
Epoch 1/2
12958/12958 [==============================] - 0s 33us/sample - loss: 7.9478 - acc: 0.5015
Epoch 2/2
3240/3240 [==============================] - 0s 13us/sample - loss: 8.0647 - acc: 0.4941
Epoch 1/2
12958/12958 [==============================] - 0s 31us/sample - loss: 7.9663 - acc: 0.5003
Epoch 2/2
3240/3240 [==============================] - 0s 14us/sample - loss: 7.9909 - acc: 0.4988
Epoch 1/2
12959/12959 [==============================] - 0s 32us/sample - loss: 7.9337 - acc: 0.5024
Epoch 2/2
3239/3239 [==============================] - 0s 13us/sample - loss: 8.1213 - acc: 0.4906
Epoch 1/2
12959/12959 [==============================] - 0s 32us/sample - loss: 8.0272 - acc: 0.4965
Epoch 2/2
3239/3239 [==============================] - 0s 13us/sample - loss: 7.7472 - acc: 0.5140


In [64]:
ret

{'acc': [0.5024691, 0.4941358, 0.49876544, 0.4905835, 0.51404756],
 'mean': 0.500000286102295}

In [70]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(31, input_dim = 31, activation = 'relu'),
  tf.keras.layers.Dense(128, activation = 'relu'),
  tf.keras.layers.Dense(64, activation = 'relu'),
  tf.keras.layers.Dense(32, activation = 'relu'),   
  tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [71]:
hist = model.fit(X_train, y_train, epochs = 10, verbose = 1)

Epoch 1/10
12958/12958 [==============================] - 1s 74us/sample - loss: 8.0379 - acc: 0.5013
Epoch 2/10
12958/12958 [==============================] - 1s 47us/sample - loss: 8.0379 - acc: 0.5013
Epoch 3/10
12958/12958 [==============================] - 1s 46us/sample - loss: 8.0379 - acc: 0.5013
Epoch 4/10
12958/12958 [==============================] - 1s 47us/sample - loss: 8.0379 - acc: 0.5013
Epoch 5/10
12958/12958 [==============================] - 1s 47us/sample - loss: 8.0379 - acc: 0.5013
Epoch 6/10
12958/12958 [==============================] - 1s 46us/sample - loss: 8.0379 - acc: 0.5013
Epoch 7/10
12958/12958 [==============================] - 1s 48us/sample - loss: 8.0379 - acc: 0.5013
Epoch 8/10
12958/12958 [==============================] - 1s 46us/sample - loss: 8.0379 - acc: 0.5013
Epoch 9/10
12958/12958 [==============================] - 1s 49us/sample - loss: 8.0379 - acc: 0.5013
Epoch 10/10
12958/12958 [==============================] - 1s 48us/sample - loss: 